This tutorial aims to run IGM providing an RGI ID

In [ ]:
!git clone https://github.com/jouvetg/igm2
!mv igm2/igm .
!rm -rf igm2
!pip install netCDF4 oggm rasterio mayavi

In [ ]:
#!/usr/bin/env python3

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import igm

 ## add custumized smb function
def params_mysmb(parser):
    parser.add_argument("--meanela", type=float, default=3000 )

def init_mysmb(params,state):
    params.meanela = np.quantile(state.usurf[state.thk>10],0.2)

def update_mysmb(params,state):
    # perturabe the ELA with sinusional signal
    ELA = ( params.meanela - 750*math.sin((state.t/100)*math.pi) )
    # compute smb linear with elevation with 2 acc & abl gradients
    state.smb  = state.usurf - ELA
    state.smb *= tf.where(state.smb<0, 0.005, 0.009)
    # cap smb by 2 m/y
    state.smb  = tf.clip_by_value(state.smb, -100, 2)
    # make sure the smb is not positive outside of the mask to prevent overflow
    if hasattr(state, "icemask"):
        state.smb  = tf.where((state.smb<0)|(state.icemask>0.5),state.smb,-10)

def final_mysmb(params,state):
    pass

# make sure to make these function new attributes of the igm module
igm.params_mysmb  = params_mysmb
igm.init_mysmb    = init_mysmb
igm.update_mysmb  = update_mysmb
igm.final_mysmb   = final_mysmb

# Select one OPTION btw the first, keep the MANDATORY ones, un/comment OPTIONAL modules
modules = [
           "prepare_data",          # OPTION 1  : download and prepare the data with OGGM
#            "optimize",             # OPTIONAL  : optimize unobservable variables from obs.
            "mysmb",                # OPTIONAL  : custom surface mass balance model
            "flow_dt_thk",          # MANDATORY : update the ice thickness solving mass cons.
            "vertical_iceflow",     # OPTIONAL  : retrieve vertical ice flow from horiz.
#           "particles",            # OPTIONAL  : seed and update particle trajectories
#           "write_particles",      # OPTIONAL  : write particle trajectories to a csv file
            "write_ncdf_ex",        # OPTIONAL  : write 2d state data to netcdf files
            "write_tif_ex",        # OPTIONAL  : write the result in tif files
#           "write_ncdf_ts",        # OPTIONAL  : write time serie data to netcdf files
#           "write_plot2d",         # OPTIONAL  : write 2d state plots to png files
            "print_info",           # OPTIONAL  : print basic live-info about the model state
            "print_all_comp_info",  # OPTIONAL  : report information about computation time
            "anim3d_from_ncdf_ex"   # OPTIONAL  : make a nice 3D animation of glacier evolution
          ]

# Collect and parse all the parameters of all model components
parser = igm.params_core()
for module in modules:
    getattr(igm, "params_" + module)(parser)
params = parser.parse_args(args=[])

# Override parameters
params.RGI         = 'RGI60-11.01238'
params.tstart      = 2000.0
params.tend        = 2100.0
params.tsave       = 5
params.plot_live   = True
params.observation = ("optimize" in modules)

# Define a state class/dictionnary that contains all the data
state = igm.State()
igm.init_state(params, state)

# Place the computation on your device GPU ('/GPU:0') or CPU ('/CPU:0')
with tf.device("/GPU:0"):

    # Initialize all the model components in turn
    for module in modules:
        getattr(igm, "init_" + module)(params, state)

    # Time loop, perform the simulation until reaching the defined end time
    while state.t < params.tend:

        # Update each model components in turn
        for module in modules:
            getattr(igm, "update_" + module)(params, state)

    # Finalize each module in turn
    for module in modules:
        getattr(igm, "final_" + module)(params, state)
